In [7]:
import urllib
from lxml import etree

# Auto ingestion of SRA metadata

1. Get a list of SRA UIDs to scrape
    1. get all publically accessible SRA (SRX) accessions
    2. remove SRA UIDs already been ingested into MetaSeek DB (all 'source_db_uid' where 'source_db' = 'SRA'
    
2. Split UIDs to scrape into batches of 500

3. For each batch, scrape metadata:
    1. efetch scrape SRA
    2. elink to BioSample, Pubmed, Nuccore
    3. efetch scrape BioSample, if exists
    4. efetch scrape Pubmed, if exists
    5. efetch scrape nuccore, if exists
    
4. Merge redundant cols to MIxS-compliant MetaSeek fields (separate script)

5. Parse fields with controlled vocabularies to MIxS CV, if possible (NOTE: future error parsing)

6. Insert scraped data into MetaSeek DB directly

# 1.A - Get a list of SRA UIDs to scrape

* find out how many publicly available samples there are; make list retstarts to use 
* Call esearch api, with rotating retstarts, to get full UID list

NOTE - most comprehensive API call to find ALL the SRA samples I can find (since an empty search term returns no accessions, not all) is term=public&field=ACS; returns all publically accessible SRA UIDs

In [58]:
def get_retstart_list(url):
    #define retstarts need for get_uid_list eutilities requests - since can only get 100,000 at a time, need to make multiple queries to get total list
    #find out count of UIDs going to pull from SRA
    g = urllib.urlopen(url)
    count_tree = etree.parse(g)
    g.close()
    count_xml = count_tree.getroot()
    num_uids = count_xml.findtext("Count")
    print 'number of publicly available UIDs in SRA: %s' % num_uids
    num_queries = 1+int(num_uids)/100000  #number of queries to do, with shifting retstart
    retstart_list = [i*100000 for i in range(num_queries)]
    print 'retstarts to use: %s' % retstart_list
    return retstart_list

def get_uid_list(ret_list):
    #scrape UIDs into list
    uid_list = []
    for retstart in ret_list:
        f = urllib.urlopen('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=sra&term=public&field=ACS&tool=metaseq&email=metaseekcloud%40gmail.com&retmax=100000&retstart='+str(retstart))
        uid_tree = etree.parse(f)
        f.close()
        uid_xml = uid_tree.getroot()
        print "appending %s accessions" % len(uid_xml.find("IdList").findall("Id"))
        #add uids to list of accessions
        for id in uid_xml.find("IdList").iterchildren():
            value = id.text
            uid_list.append(value)
    return uid_list

In [59]:
retstart_list = get_retstart_list(url='https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=sra&term=public&field=ACS&rettype=count&tool=metaseq&email=metaseekcloud%40gmail.com')

number of publicly available UIDs in SRA: 2694344
retstarts to use: [0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, 1100000, 1200000, 1300000, 1400000, 1500000, 1600000, 1700000, 1800000, 1900000, 2000000, 2100000, 2200000, 2300000, 2400000, 2500000, 2600000]


In [ ]:
uid_list = get_uid_list(ret_list=retstart_list)

# TODO 1.B - Remove SRA UIDs already in in MetaSeek DB

db_source_uid where db_source = 'SRA'

# 2 - Split UIDs to scrape into batches of 500

In [65]:
def get_batches(uid_list):
    starts = range(0,len(uid_list),500)
    ends = range(500,len(uid_list),500)
    ends.append(len(uid_list))
    batches = [list(a) for a in zip(starts, ends)]
    return batches

In [66]:
batches = get_batches(uid_list)

In [67]:
print len(batches)
print batches[0:10]
print batches[-10:]


5389
[[0, 500], [500, 1000], [1000, 1500], [1500, 2000], [2000, 2500], [2500, 3000], [3000, 3500], [3500, 4000], [4000, 4500], [4500, 5000]]
[[2689500, 2690000], [2690000, 2690500], [2690500, 2691000], [2691000, 2691500], [2691500, 2692000], [2692000, 2692500], [2692500, 2693000], [2693000, 2693500], [2693500, 2694000], [2694000, 2694344]]


# 3.A - Efetch scrape SRA metadata

In [105]:
def get_srx_metadata(batch_uid_list):
    srx_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=sra&tool=metaseq&email=metaseekcloud%40gmail.com&id='+str(batch_uid_list)[1:-1]
    s = urllib.urlopen(srx_url)
    sra_tree = etree.parse(s)
    s.close()
    sra_xml = sra_tree.getroot()

    sra_samples = sra_xml.findall("EXPERIMENT_PACKAGE")
    sdict = {}

    for which,sra_sample in enumerate(sra_samples): #the order of experiment_packages ARE in order of sra ids given - that's good
        srx_dict = {}

        sdict_id = str(batch_uid_list[which])
        srx_dict['db_source_uid'] = sdict_id
        srx_dict['db_source'] = 'SRA'
        srx_dict['expt_link'] = "https://www.ncbi.nlm.nih.gov/sra/"+str(sdict_id)

        #There are 7 top tag groups. Have to scrape data a little different for each: ['EXPERIMENT','SUBMISSION','Organization','STUDY','SAMPLE','Pool','RUN_SET']

        ###EXPERIMENT -
        if sra_sample.find("EXPERIMENT").find("IDENTIFIERS").findtext("PRIMARY_ID") is not None:
            srx_dict['expt_id'] = sra_sample.find("EXPERIMENT").find("IDENTIFIERS").findtext("PRIMARY_ID")
        if sra_sample.find("EXPERIMENT").findtext("TITLE") is not None:
            srx_dict['expt_title'] = sra_sample.find("EXPERIMENT").findtext("TITLE")
        if sra_sample.find("EXPERIMENT").find("STUDY_REF").find("IDENTIFIERS") is not None:
            if sra_sample.find("EXPERIMENT").find("STUDY_REF").find("IDENTIFIERS").findtext("PRIMARY_ID") is not None:
                srx_dict["project_id"] = sra_sample.find("EXPERIMENT").find("STUDY_REF").find("IDENTIFIERS").findtext("PRIMARY_ID")   
        if sra_sample.find("EXPERIMENT").find("DESIGN").findtext("DESIGN_DESCRIPTION") is not None:
            srx_dict['expt_design_description'] = sra_sample.find("EXPERIMENT").find("DESIGN").findtext("DESIGN_DESCRIPTION")
        if sra_sample.find("EXPERIMENT").find("DESIGN").find("SAMPLE_DESCRIPTOR").find("IDENTIFIERS").findtext("PRIMARY_ID") is not None:
            srx_dict['sample_id'] = sra_sample.find("EXPERIMENT").find("DESIGN").find("SAMPLE_DESCRIPTOR").find("IDENTIFIERS").findtext("PRIMARY_ID")
        if sra_sample.find("EXPERIMENT").find("DESIGN").find("LIBRARY_DESCRIPTOR").findtext("LIBRARY_NAME") is not None:
            srx_dict['library_name'] = sra_sample.find("EXPERIMENT").find("DESIGN").find("LIBRARY_DESCRIPTOR").findtext("LIBRARY_NAME")
        if sra_sample.find("EXPERIMENT").find("DESIGN").find("LIBRARY_DESCRIPTOR").findtext("LIBRARY_STRATEGY") is not None:
            srx_dict['library_strategy'] = sra_sample.find("EXPERIMENT").find("DESIGN").find("LIBRARY_DESCRIPTOR").findtext("LIBRARY_STRATEGY")
        if sra_sample.find("EXPERIMENT").find("DESIGN").find("LIBRARY_DESCRIPTOR").findtext("LIBRARY_SOURCE").lower() is not None:
            srx_dict['library_source'] = sra_sample.find("EXPERIMENT").find("DESIGN").find("LIBRARY_DESCRIPTOR").findtext("LIBRARY_SOURCE").lower()
        ###change library_selection to MIxS field library_screening_strategy (cv for SRA, not for MIxS)
        if sra_sample.find("EXPERIMENT").find("DESIGN").find("LIBRARY_DESCRIPTOR").findtext("LIBRARY_SELECTION") is not None:
            srx_dict['library_screening_strategy'] = sra_sample.find("EXPERIMENT").find("DESIGN").find("LIBRARY_DESCRIPTOR").findtext("LIBRARY_SELECTION")
        ###change library_layout to MIxS field library_construction_method - cv single | paired
        if sra_sample.find("EXPERIMENT").find("DESIGN").find("LIBRARY_DESCRIPTOR").findtext("LIBRARY_LAYOUT") is not None:
            srx_dict['library_construction_method'] sra_sample.find("EXPERIMENT").find("DESIGN").find("LIBRARY_DESCRIPTOR").find("LIBRARY_LAYOUT").getchildren()[0].tag.lower()
        if sra_sample.find("EXPERIMENT").find("DESIGN").find("LIBRARY_DESCRIPTOR").findtext("LIBRARY_CONSTRUCTION_PROTOCOL") is not None:
            srx_dict['library_construction_protocol'] = sra_sample.find("EXPERIMENT").find("DESIGN").find("LIBRARY_DESCRIPTOR").findtext("LIBRARY_CONSTRUCTION_PROTOCOL")
        ###change platform to MIxS field sequencing_method - cv in SRA (not in MIxS)
        if sra_sample.find("EXPERIMENT").find("PLATFORM").getchildren() is not None:
            srx_dict['sequencing_method'] = sra_sample.find("EXPERIMENT").find("PLATFORM").getchildren()[0].tag.lower()
            if sra_sample.find("EXPERIMENT").find("PLATFORM").getchildren()[0].findtext("INSTRUMENT_MODEL") is not None:
                srx_dict['instrument_model'] = sra_sample.find("EXPERIMENT").find("PLATFORM").getchildren()[0].findtext("INSTRUMENT_MODEL")
        
        ###SUBMISSION - just need the submission id
        if sra_sample.find("SUBMISSION").find("IDENTIFIERS").findtext("PRIMARY_ID") is not None:
            srx_dict['submission_id'] = sra_sample.find("SUBMISSION").find("IDENTIFIERS").findtext("PRIMARY_ID")

        ###Organization - name, address, and contact
        if sra_sample.find("Organization").findtext("Name") is not None:
            srx_dict['organization_name'] = sra_sample.find("Organization").findtext("Name")
        if sra_sample.find("Organization").find("Address") is not None:
            address = ''
            for line in sra_sample.find("Organization").find("Address").iterchildren():
                address = address+line.text+', '
            address = address[:-2]
            srx_dict['organization_address'] = address
        if len(sra_sample.find("Organization").findall("Contact"))>0:
            contacts = []
            for contact in sra_sample.find("Organization").findall("Contact"):
                name = contact.find("Name").find("First").text+' '+contact.find("Name").find("Last").text
                email = contact.get('email')
                contacts.append(name+', '+email)
            srx_dict['organization_contacts'] = contacts
        
        ###STUDY -
        if sra_sample.find("STUDY").find("IDENTIFIERS").findtext("PRIMARY_ID") is not None: 
            srx_dict['study_id'] = sra_sample.find("STUDY").find("IDENTIFIERS").findtext("PRIMARY_ID")
        if sra_sample.find("STUDY").find("DESCRIPTOR").findtext("STUDY_TITLE") is not None:
            srx_dict['study_title'] = sra_sample.find("STUDY").find("DESCRIPTOR").findtext("STUDY_TITLE")
        ###rename existing_study_type to study_type
        if sra_sample.find("STUDY").find("DESCRIPTOR").find("STUDY_TYPE") is not None:
            if sra_sample.find("STUDY").find("DESCRIPTOR").find("STUDY_TYPE").get("existing_study_type")=="Other":
                srx_dict['study_type'] = 'Other'
                if sra_sample.find("STUDY").find("DESCRIPTOR").find("STUDY_TYPE").get("add_study_type") is not None:
                    srx_dict['study_type_other'] sra_sample.find("STUDY").find("DESCRIPTOR").find("STUDY_TYPE").get("add_study_type")
            else:
                srx_dict['study_type'] sra_sample.find("STUDY").find("DESCRIPTOR").find("STUDY_TYPE").get("existing_study_type")
        if sra_sample.find("STUDY").find("DESCRIPTOR").findtext("STUDY_ABSTRACT"):
            srx_dict['study_abstract'] = sra_sample.find("STUDY").find("DESCRIPTOR").findtext("STUDY_ABSTRACT")
        
        if sra_sample.STUDY.IDENTIFIERS.PRIMARY_ID is not None:
            srx_dict['sra_study_id'] = sra_sample.STUDY.IDENTIFIERS.PRIMARY_ID.get_text()
        if sra_sample.STUDY.DESCRIPTOR.STUDY_TYPE['existing_study_type'] is not None:
            srx_dict['study_type'] = sra_sample.STUDY.DESCRIPTOR.STUDY_TYPE['existing_study_type']
        if len(sra_sample.STUDY.DESCRIPTOR.find_all(True,recursive=False))>0:
            for tag in sra_sample.STUDY.DESCRIPTOR.find_all(True,recursive=False):
                srx_dict['sra_'+tag.name.lower()] = tag.get_text()

        ###SAMPLE - just need sample id
        if sra_sample.SAMPLE.IDENTIFIERS.PRIMARY_ID is not None:
            srx_dict['sra_sample_id'] = sra_sample.SAMPLE.IDENTIFIERS.PRIMARY_ID.get_text()

        ###Pool - skip, redundant

        ###RUN_SET - record total num_runs, for each run basic stats
        if len(sra_sample.RUN_SET.find_all('RUN'))>0:
            srx_dict['num_runs_per_sra'] = len(sra_sample.RUN_SET.find_all('RUN'))
            for run_ix,run in enumerate(sra_sample.RUN_SET.find_all('RUN')):
                srx_dict['sra_run'+str(run_ix)+'_id'] = run['accession']
                srx_dict['sra_run'+str(run_ix)+'_total_spots'] = run['total_spots']
                srx_dict['sra_run'+str(run_ix)+'_total_bases'] = run['total_bases']
                srx_dict['sra_run'+str(run_ix)+'_size'] = run['size']
                if run.Bases.find_all('Base') is not None:
                    for base in run.Bases.find_all('Base'):
                        srx_dict['sra_run'+str(run_ix)+'_base'+base['value']+'_count'] = base['count']

        sdict[sdict_id] = srx_dict

    #destroy sra_xml, we're done with it
    sra_xml.decompose()
    return sdict


IndentationError: expected an indented block (<ipython-input-105-d7cb34e528e5>, line 28)

In [86]:
batch = batches[0]
batch_uid_list = map(int,uid_list[batch[0]:batch[1]])
batch_uid_list[0:10]

[4123567,
 4123566,
 4123565,
 4123562,
 4123561,
 4123560,
 4123559,
 4123556,
 4123555,
 4123554]

In [96]:
batch_uid_list[0]

4123567

In [91]:
srx_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=sra&tool=metaseq&email=metaseekcloud%40gmail.com&id='+str(batch_uid_list)[1:-1][0:16] #leave out 
srx_url

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=sra&tool=metaseq&email=metaseekcloud%40gmail.com&id=4123567, 4123566'

In [92]:
s = urllib.urlopen(srx_url)
sra_tree = etree.parse(s)
s.close()
sra_xml = sra_tree.getroot()

In [94]:
sra_xml.getchildren()

[<Element EXPERIMENT_PACKAGE at 0x109655830>,
 <Element EXPERIMENT_PACKAGE at 0x1096558c0>]

In [103]:
sra_samples = sra_xml.findall("EXPERIMENT_PACKAGE")
sdict = {}
sra_samples

NameError: name 'srx_dict' is not defined

In [98]:
sra_sample = sra_samples[0]
sra_sample

<Element EXPERIMENT_PACKAGE at 0x109655830>

In [100]:
sra_sample.find("EXPERIMENT").getchildren()

[<Element IDENTIFIERS at 0x109655d40>,
 <Element TITLE at 0x109655488>,
 <Element STUDY_REF at 0x1040ba6c8>,
 <Element DESIGN at 0x1040ba440>,
 <Element PLATFORM at 0x1040ba908>,
 <Element PROCESSING at 0x1040bab90>]

In [182]:
study_links = {}
for study_link in sra_sample.find("STUDY").find("STUDY_LINKS").iterchildren():
    if study_link.find("XREF_LINK") is not None:
        study_links[study_link.find("XREF_LINK").findtext("DB")] = study_link.find("XREF_LINK").findtext("ID")
    if study_link.find("URL_LINK") is not None:
        study_links[study_link.find("URL_LINK").findtext("LABEL")] = study_link.find("URL_LINK").findtext("URL")
study_links

{"CFSAN's Whole Genome Sequencing program": 'http://www.fda.gov/Food/FoodScienceResearch/WholeGenomeSequencingProgramWGS/default.htm',
 'pubmed': '25540336'}